## Read Me

The following code works with the 25% sample dataset, named 'train_data_cleaned_final.csv.zip' on GCP
- Adds distance to the 25% sample dataset
- Cleaned out distance that is longer than 100km
- recoded timestamp into year, month, day, hour, date
- filter only 2013-2016 data
- Saved the dataset as 'ranged_data.csv" which will be the dataset to work on for following analysis

## Adding distance, year, month, day, hour, date

In [1]:
# Dependencies
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import calendar
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from datetime import datetime
import warnings
import scipy.stats as stats
warnings.filterwarnings('ignore')
%matplotlib inline


#gcp dependencies
from google.cloud import storage
from io import BytesIO
client = storage.Client()
bucket = client.get_bucket("nytaxi_mz")

In [2]:
#Reading the cleaned 25% sample data from csv to use forward
blob = storage.blob.Blob("train_data_cleaned_final.csv.zip",bucket)
content = blob.download_as_string()
data_df = pd.read_csv(BytesIO(content),compression='zip', header=0, sep=',', quotechar='"')

In [3]:
#Understanding the data structure before we moved on
data_df.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
13497798,13807164,13855959,20915807,2011-03-16 08:35:16.0000004,3.7,2011-03-16 08:35:16 UTC,-73.992502,40.738154,-73.998564,40.735588,2
13497799,13807165,13855960,27716687,2014-12-12 20:03:47.0000005,29.0,2014-12-12 20:03:47 UTC,-73.863911,40.769995,-73.977236,40.764737,1
13497800,13807166,13855961,29517547,2015-02-19 16:16:35.0000007,8.0,2015-02-19 16:16:35 UTC,-73.962456,40.799751,-73.977676,40.786709,1
13497801,13807167,13855962,34958411,2011-01-29 01:09:50.0000004,11.3,2011-01-29 01:09:50 UTC,-73.985784,40.731165,-73.949620,40.706679,1
13497802,13807168,13855963,30755059,2013-10-27 22:42:11.0000002,15.5,2013-10-27 22:42:11 UTC,-73.994586,40.750314,-73.949266,40.777074,1


In [4]:
#recode pickup_datetime into time stamp
data_df['new_time'] = (data_df['pickup_datetime']).apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z"))

#clean up the dataframe
data_df = data_df.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime'], axis=1)

#Our weather data only includes data from 2013 - 2015, so in our sampled data,
#we also filtered out data from the other years
time_range = (data_df['new_time'] >= "2013-01-01") & (data_df['new_time'] <= "2015-12-31")
ranged_data = data_df.loc[time_range]

In [5]:
#calculate the distance between pick-up and drop-off spot in kilometer


def distance_from_coord(lat1, lon1, lat2, lon2):
    radius = 6371  # km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return d

def distance(row):

    lat1, lon1 = row['pickup_latitude'], row['pickup_longitude']
    lat2, lon2 = row['dropoff_latitude'], row['dropoff_longitude']
    return distance_from_coord(lat1, lon1, lat2, lon2)

#apply the distance calculation function into the dataframe
ranged_data['distance'] = data_df.apply(distance, axis=1)

In [7]:
#Clean out travel distance that is more than 100 km (data analysis is limited in NYC)
ranged_data = ranged_data[(ranged_data['distance'] <= 100) & (ranged_data['distance']>0)]

In [8]:
#We separate the timestamp data into year, hour, weekday, and date for future time series analayis
ranged_data['date'] = ranged_data['new_time'].apply(lambda x: x.date())
ranged_data['year'] = ranged_data['new_time'].apply(lambda x: x.year)
ranged_data['month'] = ranged_data['new_time'].apply(lambda x: x.month)
ranged_data['day'] = ranged_data['new_time'].apply(lambda x: calendar.day_name[x.weekday()])
ranged_data['hour_of_day'] = ranged_data['new_time'].apply(lambda x: x.hour)

In [9]:
#display the data
ranged_data.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,new_time,distance,airport_ride,date,year,month,day,hour_of_day
3,8.0,-73.963240,40.766012,-73.944857,40.778990,2,2013-02-16 13:48:00,2.116328,False,2013-02-16,2013,2,Saturday,13
4,9.5,-73.973102,40.744402,-73.972842,40.760473,1,2013-10-02 09:31:29,1.787148,False,2013-10-02,2013,10,Wednesday,9
10,8.5,-73.996201,40.721088,-73.978867,40.723999,1,2015-04-16 20:18:51,1.496196,False,2015-04-16,2015,4,Thursday,20
13,19.0,-74.001901,40.715667,-73.972271,40.791520,1,2013-12-19 00:11:42,8.795986,False,2013-12-19,2013,12,Thursday,0
14,5.0,-73.969292,40.749357,-73.970087,40.756832,3,2014-04-29 19:27:00,0.833875,False,2014-04-29,2014,4,Tuesday,19


In [10]:
#display descriptives
ranged_data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,year,month,hour_of_day
count,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06,5.043754e+06
mean,1.272773e+01,-7.397510e+01,4.075058e+01,-7.397440e+01,4.075103e+01,1.700874e+00,3.414870e+00,2.013770e+03,5.883276e+00,1.351972e+01
std,1.065598e+01,3.516916e-02,2.710554e-02,3.392219e-02,3.104374e-02,1.362558e+00,3.694512e+00,7.413974e-01,3.394984e+00,6.521970e+00
min,2.500000e+00,-7.425878e+01,4.048211e+01,-7.425877e+01,4.047923e+01,1.000000e+00,8.406675e-05,2.013000e+03,1.000000e+00,0.000000e+00
25%,6.500000e+00,-7.399231e+01,4.073636e+01,-7.399159e+01,4.073513e+01,1.000000e+00,1.283600e+00,2.013000e+03,3.000000e+00,9.000000e+00
50%,9.500000e+00,-7.398210e+01,4.075318e+01,-7.398050e+01,4.075366e+01,1.000000e+00,2.184734e+00,2.014000e+03,5.000000e+00,1.400000e+01
75%,1.450000e+01,-7.396815e+01,4.076749e+01,-7.396466e+01,4.076840e+01,2.000000e+00,3.981279e+00,2.014000e+03,9.000000e+00,1.900000e+01
max,4.500000e+02,-7.370054e+01,4.091736e+01,-7.370031e+01,4.091753e+01,9.000000e+00,4.522777e+01,2.015000e+03,1.200000e+01,2.300000e+01


In [ ]:
#save the final data into zip file for future use
ranged_data.to_csv("ranged_data.csv")